# aid-iptables

aid-iptables is a command line utility that can query the ISP AID List and automatically create iptables rules to block ips on the AID List.

In [4]:
aid-iptables -h

Usage: aid-iptables [OPTIONS]

Options:
  --start-date TEXT          default='1 week ago' - Generate AID list with IPs
                             detected since start-date
  --service TEXT             default= (all services) - Only include hits for
                             the specified service
  --whitelist PATH           Path to whitelist file containing one ip address
                             or subnet per line
  --seen-count INTEGER       default=10 - Minimum # of alerts an aid list IP
                             has generated
  --chain-name TEXT          default=aid - The name of the iptables chain to
                             use for the aid list
  --input-chain-pos INTEGER  default=0 - Position in INPUT chain to add a jump
                             to the aid chain
  -h, --help                 Show this message and exit.


### Default behavior

#### Running **aid-iptables** with no arguments will:

- Query the AID List for all entries added within the last 7 days
- create (if needed) an IPTables Chain named "aid"
- add an unconditional jump to the "aid" chain as the first rule in the IPTables INPUT chain
- For each entry in the returned AID List, add an IPTables rule, with target=DROP, to the "aid" chain

In [1]:
# IPTables before running aid-iptables
iptables -L

Chain INPUT (policy ACCEPT)
target     prot opt source               destination         

Chain FORWARD (policy DROP)
target     prot opt source               destination         

Chain OUTPUT (policy ACCEPT)
target     prot opt source               destination         


In [4]:
# Run api-iptables with default settings
aid-iptables

In [33]:
# Truncating output for space
iptables -L -n | head -n 16

Chain INPUT (policy ACCEPT)
target     prot opt source               destination         
aid        all  --  0.0.0.0/0            0.0.0.0/0           

Chain FORWARD (policy DROP)
target     prot opt source               destination         

Chain OUTPUT (policy ACCEPT)
target     prot opt source               destination         

Chain aid (1 references)
target     prot opt source               destination         
DROP       all  --  1.93.59.153          0.0.0.0/0           
DROP       all  --  5.45.79.24           0.0.0.0/0           
DROP       all  --  23.96.45.195         0.0.0.0/0           
DROP       all  --  27.221.10.43         0.0.0.0/0           


### Changing the time window 

By default, aid-iptables queries for IP address that have been added to the AID list within the last 7 days.  The **start-date** parameter can be used to query for a shorter or longer window.  **start-date** accepts most date formats and english descriptions.

    aid-iptables --start-date="1 month"
    aid-iptables --start-date="11/12/2015"

### Filtering by service

#### To filter the AID list to only include IPs that have been detected attacking a specific service
    
    aid-iptables --service=ssh

#### To filter by multiple services, either pass a comma seperated list to --service, or invoke the --service option multiple times

    aid-iptables --service=ssh --service=rdp
    aid-iptables --service=ssh,rdp

### Using a whitelist

A whitelist, a text file containing one IP Addresses or Subnet in CIDR notation per line, can be used to prevent IPTables block rules from being created.  

    aid-iptables --whitelist=<path-to-whitelist>


In [25]:
aid-iptables
iptables -L aid -n | head -n 6

Chain aid (1 references)
target     prot opt source               destination         
DROP       all  --  1.93.56.129          0.0.0.0/0           
DROP       all  --  1.93.59.153          0.0.0.0/0           
DROP       all  --  5.45.79.24           0.0.0.0/0           
DROP       all  --  14.160.13.174        0.0.0.0/0           


In [26]:
cat ~/whitelist

1.93.56.0/24
14.160.13.174


In [29]:
aid-iptables --whitelist=/root/whitelist
iptables -L aid -n | head -n 6

Chain aid (1 references)
target     prot opt source               destination         
DROP       all  --  1.93.59.153          0.0.0.0/0           
DROP       all  --  5.45.79.24           0.0.0.0/0           
DROP       all  --  23.96.45.195         0.0.0.0/0           
DROP       all  --  27.221.10.43         0.0.0.0/0           


### Filtering by Aggresiveness

The AID list keeps track of how many times an aggressive IP address has been generated an alert. The **seen-count** argument can be used to set minimum number of detections for an IP address to be blocked.  The default for **seen_count** is 10.

#### Block every IP address that has generated an alert in the past month
    aid-iptables --seen_count=1 --start-date="1 month"
    
#### Only block IP addresses that have generated at least 50 alerts in the past month
    aid-iptables --seen_count=50 --start-date="1 month"


### IPTables Options

### chain-name

The **chain-name** parameter controls the name of the IPtables chain used by aid-iptables.  By default, the chain's name is aid.  Caution must be taken if this parameter is changed after aid-iptables has been run, as it will not attempt to flush or remove the old chain, this must be done manually

### input-chain-pos 

By default, aid-iptables will insert an uncoditional jump to **chain-name** as the first rule in the INPUT chain.  The postion of this unconditional jump can be changed with **input-chain-pos** argument